## Задача поиск похожих по эмбеддингам

что надо сделать 

2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов

на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу.

3. Проверить насколько хорошо работают подходы

In [3]:
# !wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [4]:
# !pip install corus

In [5]:
from corus import load_lenta
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)

In [6]:
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [25]:
# !pip install pymorphy2 stop-words annoy

In [8]:
import pandas as pd
import numpy as np

import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
from sklearn.model_selection import train_test_split
import annoy
import re
from tqdm import tqdm
from tqdm import tqdm_notebook

import gensim.models
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

2023-12-09 13:05:29.035625: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-09 13:05:29.074326: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-09 13:05:29.378714: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-09 13:05:29.380877: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-09 13:05:30.216723: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [9]:
corpus_title = []
corpus_text = []

for num, iter in tqdm(enumerate(records)):
    corpus_title.append(iter.title)
    corpus_text.append(iter.text)
    if num == 99999:
        break

99999it [00:02, 41937.31it/s]


In [10]:
corpus_title[10], corpus_text[10]

('Овечкин повторил свой рекорд',
 ' Капитан «Вашингтона» Александр Овечкин сделал хет-трик в игре Национальной хоккейной лиги (НХЛ) с «Каролиной». Таким образом он продлил результативную серию до 13 матчей и повторил личный рекорд января-февраля 2007 года. Об этом сообщает Sports.ru. Встреча завершилась со счетом 5:6 Б — «Вашингтон» переиграл «Каролину» по булитам. Для россиянина это второй подряд хет-трик. До этого он отметился им в матче против «Детройт Ред Уингс». Клубный рекорд остается за Майклом Гартнером — 17 матчей. В соперничестве бомбардиров Овечкин занял пятое место с 42 (28+14) очками в 31 матче.')

In [11]:
df = pd.DataFrame(corpus_title, columns=['title'])
df['text'] = corpus_text
df.head()

,title,text
0,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...
1,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...
2,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...
3,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...
4,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...


In [12]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [13]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [14]:
preprocess_txt(df['text'].values[0])[:10]

['австрийский',
 'правоохранительный',
 'орган',
 'представить',
 'доказательство',
 'нарушение',
 'российский',
 'биатлонист',
 'антидопинговый',
 'правило']

In [15]:
pattern = r'[^а-яА-Я0-9]'
df['text'] = df['text'].apply(lambda x: re.sub(pattern, ' ', x))

In [16]:
sentences = []
counter = 0

for line in tqdm_notebook(df.text):
    spls = preprocess_txt(line)
    sentences.append(spls)
    counter += 1
    if counter > 100000:
        break

  0%|          | 0/100000 [00:00<?, ?it/s]

In [17]:
sentences[0][:10]

['австрийский',
 'правоохранительный',
 'орган',
 'представить',
 'доказательство',
 'нарушение',
 'российский',
 'биатлонист',
 'антидопинговый',
 'правило']

In [18]:
modelW2V = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=2)

In [19]:
modelFT = FastText(sentences=sentences, vector_size=100, min_count=2, window=5, workers=8)

In [20]:
w2v_index = annoy.AnnoyIndex(100 ,'angular')
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

for line in tqdm_notebook(df.text):
        n_w2v = 0
        n_ft = 0
        spls = line
        index_map[counter] = spls
        question = preprocess_txt(spls)

        vector_w2v = np.zeros(100)
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)

        counter += 1
        # ограничение в  100000 строк
        if counter > 100000:
            break


w2v_index.build(10)
ft_index.build(10)

  0%|          | 0/100000 [00:00<?, ?it/s]

True

In [21]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(100)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [22]:
TEXT = 'новые технологии'

In [23]:
get_response(TEXT, w2v_index, modelW2V, index_map)

[' Норникель  создаст в  Сколково  партнерский центр  который займется развитием и продвижением новых    технологий компании   Норникель  и инновационный центр подписали стратегическое соглашение о сотрудничестве во вторник  16 октября  на форуме  Открытые инновации  в Москве  Об этом сообщает корреспондент  Ленты ру   Стороны уверены  что это соглашение придаст новый импульс всей    отрасли  в целом  и привлечет интерес лучших технологических компаний страны   Существующие    инициативы уже приносят компании существенные выгоды  и мы видим большой потенциал в продолжении данной работы и расширении горизонтов    отметил старший вице президент   финансовый директор  Норникеля  Сергей Малышев  В руководстве  Сколково  подчеркнули  что ГМК уже имеет успешный опыт внедрения в производственные и бизнес процессы самых передовых технологий  Теперь эта работа будет вестись совместно с кластером передовых производственных технологий  ядерных и космических технологий инновационного центра  реали

In [24]:
get_response(TEXT, ft_index, modelFT, index_map)

['Роберт Хван                 генеральный директор тайваньской компании          которая занимается производством        в Индии  подтвердил некоторые слухи о новом смартфоне        Об этом пишет                      По его словам  новый        получит водонепроницаемый корпус  как и нынешняя модель  Помимо этого  Хванг подтвердил слухи о том  что гаджет оснастят системой беспроводной зарядки   Процесс сборки          не сильно изменился по сравнению с предыдущими поколениями  хотя новые функции  вроде водонепроницаемости и беспроводной зарядки  требуют несколько иных тестов     рассказал глава                начала производить        в Индии в мае 2017 года          на данный момент собирает лишь самые дешевые версии смартфонов                   и 6  Также компания выпускает устройства для китайских и индийских брендов  Новый        будет представлен осенью 2017 года на ежегодной презентации        В апреле были опубликованы первые фотографии макета устройства  а позже с ним записали 